In [ ]:
from openai import OpenAI

import json
from google.colab import userdata
from datetime import datetime
import re
# Set your API key

openai = OpenAI(
  api_key=userdata.get('openai'),  # this is also the default, it can be omitted
)

def extract_event_info(text):
    prompt = f"""
    Extract structured event details from the following text and return JSON:

    Text: "{text}"

    Provide only a JSON output with the following fields without give any other characters:
    {{
      "event_title": "Short title of the event",
      "start_time": "ISO 8601 format start time",
      "end_time": "ISO 8601 format end time",
      "location": "Event location",
      "reminder": "Recommended reminder time in ISO 8601",
      "importance": "low/medium/high",
      "summary": "Summary of the event"
    }}
    """

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  # Use GPT-4 or GPT-3.5
        messages=[{"role": "system", "content": "You are an AI that extracts structured event details, provide only a JSON output without give any other characters."},
                  {"role": "user", "content": prompt},],
    )

    output_text = response.choices[0].message.content

    # Convert response to JSON
    try:
        event_data = json.loads(output_text)
        event_data = {
            "event_title": event_data.get("event_title", "No title"),
            "start_time": event_data.get("start_time", datetime.now().isoformat()),
            "end_time": event_data.get("end_time", datetime.now().isoformat()),
            "location": event_data.get("location", "No location"),
            "reminder": event_data.get("reminder", datetime.now().isoformat()),
            "importance": event_data.get("importance", "medium"),
            "summary": event_data.get("summary", "No summary")
        }

    except json.JSONDecodeError:
        print("Error parsing JSON",output_text)
        return {"output_text":output_text}

    return event_data


In [ ]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/input.csv')
# Apply the event extraction function to the "text" column
event_data_list = df['text'].apply(extract_event_info)

# Convert the event data to a DataFrame
event_data_df = pd.json_normalize(event_data_list)

# Concatenate the original dataframe with the event data
final_df = pd.concat([df, event_data_df], axis=1)

# Save the new CSV with the extracted event data
final_df.to_csv('output_with_event_data.csv', index=False)


Mounted at /content/drive


In [ ]:
final_df

,text,event_title,start_time,end_time,location,reminder,importance,summary
0,The AI & Machine Learning Summit 2025 will tak...,AI & Machine Learning Summit 2025,2025-07-20T09:00:00,2025-07-20T18:00:00,"New York City, Grand Conference Hall",2025-07-20T08:00:00,high,Industry experts discussing latest trends in A...
1,Get ready for an unforgettable night! BTS is p...,BTS Live Concert,2025-09-15T19:30:00,2025-09-15T23:00:00,Seoul Olympic Stadium,2025-09-15T12:00:00,high,BTS live performance at the Seoul Olympic Stad...
2,Stanford University’s Data Science Bootcamp is...,Python Workshop,2025-08-05T10:00:00,2025-08-05T15:00:00,"Room 305, Computer Science Building, Stanford ...",2025-08-05T09:00:00,medium,Stanford University’s Data Science Bootcamp ho...
3,Acme Corp will hold its Q3 Financial Review me...,Q3 Financial Review Meeting,2025-10-10T14:00:00,None,Main conference room on the 10th floor of head...,2025-10-10T08:00:00,medium,Acme Corp Q3 Financial Review meeting for depa...
4,Join us for the official launch of the new iPh...,Official Launch of iPhone 17,2025-09-12T10:00:00,2025-09-12T12:00:00,"Apple Park, Cupertino",2025-09-12T09:00:00,high,Join the official launch event of the new iPho...
5,You're invited to Emma’s 30th birthday bash! C...,Emma’s 30th birthday bash,2025-04-05T19:00:00,2025-04-06T00:00:00,"The Blue Moon Bar, downtown Chicago",2025-03-25T00:00:00,medium,Celebrate Emma’s 30th birthday bash at The Blu...
6,Catch the action live! The LA Lakers take on t...,LA Lakers vs. Golden State Warriors,2025-02-18T20:00:00,None,Staples Center,2025-02-18T18:00:00,medium,NBA game between the LA Lakers and the Golden ...
7,We are thrilled to invite you to celebrate the...,Jessica & Michael's Wedding,2025-06-22T16:00:00,2025-06-22T23:59:59,Rosewood Garden in Napa Valley,2025-04-30T09:00:00,high,Wedding ceremony of Jessica & Michael followed...
8,Join us for an exclusive screening of 'The Fut...,The Future of AI Screening,2025-05-10T19:00:00,None,AMC Theaters in Los Angeles,2025-05-10T09:00:00,medium,Exclusive screening of 'The Future of AI' foll...
9,The Annual Hope Gala is happening on November ...,The Annual Hope Gala,2025-11-30T18:00:00,None,"The Ritz-Carlton, New York",2025-11-30T10:00:00,medium,"An evening of fine dining, live music, and auc..."


In [ ]:
# test_text = "Apple will host its annual product launch event in San Francisco on March 15, 2025. CEO Tim Cook will introduce new devices."
# output = extract_event_info(test_text)

# print(json.dumps(output, indent=2))


In [ ]:
!pip install flask flask-cors pyngrok
!ngrok config add-authtoken 2u6t2ndsF0wEj5i16ucjM7hzaJ1_82m7NogCKjeneWMnHczLS

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,235 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubun

In [ ]:
import os
import threading
from flask import Flask,Request, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route('/extract_event_info', methods=['POST'])
def extract_event_info_endpoint():
    data = request.get_json()  # Get JSON data from the POST request

    if not data or 'text' not in data:
        return jsonify({"error": "No text provided"}), 400

    event_text = data['text']
    event_info = extract_event_info(event_text)

    return jsonify(event_info)

@app.route("/", methods=["GET"])
def root():
    return '''
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTurk Event Extraction Task</title>
</head>
<body>
    <h1>Event Information Extraction Task</h1>
    <p>Please review the AI-generated event data and make necessary adjustments before submitting.</p>

    <crowd-form>
        <!-- 显示原始文本，AI 处理后 -->
        <h2>Original Text</h2>
        <crowd-textarea id="originalText" name="originalText" rows="6" required></crowd-textarea>

        <h2>Review and Adjust AI Generated Event Data</h2>

        <label for="eventTitle">Event Title:</label>
        <crowd-input type="text" id="eventTitle" name="eventTitle" required></crowd-input>
        <br><br>

        <label for="startTime">Start Time:</label>
        <crowd-input type="datetime-local" id="startTime" name="startTime" required></crowd-input>
        <br><br>

        <label for="endTime">End Time:</label>
        <crowd-input type="datetime-local" id="endTime" name="endTime" required></crowd-input>
        <br><br>

        <label for="location">Location:</label>
        <crowd-input type="text" id="location" name="location"></crowd-input>
        <br><br>

        <label for="reminder">Reminder Time:</label>
        <crowd-input type="datetime-local" id="reminder" name="reminder"></crowd-input>
        <br><br>

        <label for="importance">Importance:</label>
        <select id="importance" name="importance" required>
            <option value="high">High</option>
            <option value="medium">Medium</option>
            <option value="low">Low</option>
        </select>
        <br><br>

        <label for="summary">Event Summary:</label>
        <crowd-textarea id="summary" name="summary" rows="4" required></crowd-textarea>
        <br><br>

        <button type="submit">Submit Adjustments</button>
    </crowd-form>
</body>
</html>
'''


app.run()  # Don't use reloader in Colab

 * ngrok tunnel "https://f6ca-34-75-97-175.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 21:08:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 21:08:07] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# from flask_ngrok import run_with_ngrok
# from flask import Flask, render_template , request, jsonify
# # import os
# # from google.colab import drive
# # drive.mount('/content/gdrive')

# # PEOPLE_FOLDER = os.path.join('static', 'people_photo')
# # app = Flask(__name__, template_folder='/content/static')
# # app.config['UPLOAD_FOLDER'] = PEOPLE_FOLDER
# app = Flask(__name__)
# run_with_ngrok(app)
# app.config['DEBUG'] = True
# @app.route('/extract_event_info', methods=['POST'])
# def extract_event_info_endpoint():
#     data = request.get_json()  # Get JSON data from the POST request

#     if not data or 'text' not in data:
#         return jsonify({"error": "No text provided"}), 400

#     event_text = data['text']
#     event_info = extract_event_info(event_text)

#     return jsonify(event_info)

# app.run()  # Don't use reloader in Colab